In [1]:
import os  
import sys  
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import tensorflow as tf  
from keras.backend.tensorflow_backend import set_session  
from keras.utils import multi_gpu_model
config = tf.ConfigProto()  
config.gpu_options.allow_growth = True  
set_session(tf.Session(config=config))  
RAW_FILE_PATH = '../data/'

/home/user/cuser/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/user/cuser/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import numpy as np
import pandas as pd
import keras.backend as K
from keras import layers
from keras.layers import Dense
from keras.optimizers import Adam
from keras.layers import Input, Embedding, Reshape, Add, Dropout
from keras.layers import Flatten, merge, Lambda
from keras.layers.normalization import BatchNormalization
from keras.models import Model
# from keras.utils.vis_utils import plot_model
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from keras.callbacks import EarlyStopping
import random
from keras.callbacks import History
from keras.utils import multi_gpu_model
import keras
import pickle

In [3]:
import seaborn as sns
# from sklearn.model_selection import train_test_split
import gc
import os
from sklearn.metrics import roc_auc_score
from collections import Counter
import matplotlib.pyplot as plt

%matplotlib inline
#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

import warnings
warnings.filterwarnings("ignore")

#中文字体
import matplotlib
matplotlib.use('qt4agg')
#指定默认字体
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family'] = 'sans-serif'
#解决负号'-'显示为方块的问题
matplotlib.rcParams['axes.unicode_minus'] = False

In [4]:
fnl_seed = 2018
np.random.seed(fnl_seed)
os.environ['PYTHONHASHSEED'] = '2018'
random.seed(fnl_seed)
tf.set_random_seed(fnl_seed)

In [5]:
def read_df_float32(file_path, read_cols=[], sep=',', nrows=None, exclude_cols=[], skip_nrows=None, verbose=False):
    assert((len(read_cols) == 0) | (len(exclude_cols) == 0))
    
    if verbose: print('reading', file_path)
    if len(read_cols) > 0:
        data = pd.read_csv(file_path, nrows=1, sep=sep, usecols=read_cols)
    else:
        data = pd.read_csv(file_path, nrows=1, sep=sep)

    
    read_type_dict = {}
    for col in data.columns[data.dtypes == np.float64]:
            read_type_dict[col] = np.float32

      
    if len(read_cols) > 0:
        data = pd.read_csv(file_path, sep=sep, usecols=read_cols, nrows=nrows, skiprows=skip_nrows)
    elif len(exclude_cols)>0:
        read_cols = filter(lambda x: x not in exclude_cols, data.columns)
        data = pd.read_csv(file_path, sep=sep, usecols=read_cols, nrows=nrows, skiprows=skip_nrows)
    else:
        data = pd.read_csv(file_path, sep=sep, nrows=nrows, skiprows=skip_nrows)
    return data

## Model

In [6]:
# 用 官方的评分方法 计算AUC
def cal_tecent_auc(data, val_pred):
    assert(type(data) == pd.core.frame.DataFrame and 'aid' in data.columns and 'uid' in data.columns)
    
    
    df_eval = data[['aid', 'label']].copy()
    assert(len(val_pred) == df_eval.shape[0])
    df_eval['pred'] = val_pred
    return np.mean(df_eval.groupby('aid').apply(lambda x:roc_auc_score(x['label'], x['pred'])))

In [7]:
def embedding_input(name, n_in, n_out):
    inp = Input(shape = (1, ), dtype = 'int64', name = name)
#     if n_out>8: n_out//=2
#     n_out = 5
    return inp, Embedding(n_in, n_out, input_length = 1, embeddings_initializer=keras.initializers.uniform(seed=fnl_seed),
                         trainable=True)(inp)

def continous_input(name):
    inp = Input(shape=(1, ), dtype = 'float32', name = name)
    return inp, Reshape((1, 1))(inp)



# similar to https://github.com/jrzaurin/Wide-and-Deep-Keras/blob/master/wide_and_deep_keras.py
def feature_generate(data_dict, cate_columns, cont_columns, wide_columns=[]):
    embeddings_tensors = []
    continuous_tensors = []
    
    for ec in cate_columns:
        layer_name = ec + '_inp'
        layer_name = layer_name.replace('+', '_')
        layer_name = layer_name.replace('%', '_')
        # For categorical features, we em-bed the features in dense vectors of dimension 6×(category cardinality)**(1/4)
        embed_dim = data_dict[ec] if int(6 * np.power(data_dict[ec], 1/4)) > data_dict[ec] else int(6 * np.power(data_dict[ec], 1/4))
#         print(ec, embed_dim)
        t_inp, t_build = embedding_input(layer_name, data_dict[ec], embed_dim)
        embeddings_tensors.append((t_inp, t_build))
        del(t_inp, t_build)
        
    for cc in cont_columns:
        layer_name = cc + '_inp'
        layer_name = layer_name.replace('+', '_')
        layer_name = layer_name.replace('%', '_')
        t_inp, t_build = continous_input(layer_name)
        continuous_tensors.append((t_inp, t_build))
        del(t_inp, t_build)
        
    for wc in wide_columns:
        layer_name = 'w_' + wc + '_inp'
        layer_name = layer_name.replace('+', '_')
        layer_name = layer_name.replace('%', '_')
        t_inp, t_build = continous_input(layer_name)
        continuous_tensors.append((t_inp, t_build))
        del(t_inp, t_build)
    
    inp_layer =  [et[0] for et in embeddings_tensors]
    inp_layer += [ct[0] for ct in continuous_tensors]
    inp_embed =  [et[1] for et in embeddings_tensors]
    inp_embed += [ct[1] for ct in continuous_tensors]
    
    return inp_layer, inp_embed

In [8]:
def fit(inp_layer, inp_embed, wide_feas_cnt=-1 ,deep_layer_sizes=[32, 32, 32], cross_layer_cnt=1, adam_r=0.05,
        drop_out_r = None, use_bn=False, grad_cliping_norm=None, is_print_sum=False, use_multi_gpu = False):
    
    if cross_layer_cnt:
        assert((type(cross_layer_cnt) == int) and cross_layer_cnt > 0)
        
    print('deep_layer_sizes:', deep_layer_sizes, ', cross_layer_cnt:',cross_layer_cnt, ', adam_r:', adam_r, 
          ', drop_out_r:',drop_out_r, ', use_bn:', use_bn, ', grad_cliping_norm:', grad_cliping_norm)
    

#     input = merge(inp_embed, mode = 'concat')
    if wide_feas_cnt > 0:
        input = merge(inp_embed[:-wide_feas_cnt], mode = 'concat')
    else:
        input = merge(inp_embed, mode = 'concat')
        
    
    input = BatchNormalization()(input) #不宜在这里做BN，因为embedding之后的数值相差很大
    print("int_shape(input) :", K.int_shape(input))
    # deep layer
    for i, node_cnt in enumerate(deep_layer_sizes):
        if i == 0:
            deep = Dense(node_cnt, activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=fnl_seed),
                        bias_initializer=keras.initializers.Constant(value=0.1))(Flatten()(input))
            if use_bn: 
                deep = BatchNormalization()(deep)
            if drop_out_r != None:
                assert(type(drop_out_r) == float)
                deep = Dropout(drop_out_r, seed=fnl_seed)(deep)
        else:
            deep = Dense(node_cnt, activation='relu', kernel_initializer=keras.initializers.glorot_uniform(seed=fnl_seed),
                        bias_initializer=keras.initializers.Constant(value=0.1))(deep)
            if use_bn:
                deep = BatchNormalization()(deep)
            if drop_out_r != None:
                assert(type(drop_out_r) == float)
                deep = Dropout(drop_out_r, seed=fnl_seed)(deep)

    # cross layer
    if cross_layer_cnt:
        cross = CrossLayer(output_dim = input.shape[2].value, num_layer = cross_layer_cnt, name = "cross_layer")(input)

    #concat both layers
    
    if cross_layer_cnt:
        if wide_feas_cnt > 0:
            wide_input = merge(inp_embed[-wide_feas_cnt:], mode = 'concat')
            output = merge([deep, cross, Flatten()(wide_input)], mode = 'concat')
        else:
            output = merge([deep, cross], mode = 'concat')
            
    else:
        output = deep
    output = Dense(1, activation = 'sigmoid', kernel_initializer=keras.initializers.glorot_uniform(seed=fnl_seed),
                        bias_initializer=keras.initializers.Constant(value=0.1))(output)
    model = Model(inp_layer, output)
    if is_print_sum: 
        print(model.summary())
#         plot_model(model, to_file = 'model.png', show_shapes = True)
    
    if use_multi_gpu:
        try:
            model = multi_gpu_model(model, gpus=4)
            print("Training using multiple GPUs..")
        except Exception as e:
            print("Training using single GPU or CPU..")
            print(e)
    
    if grad_cliping_norm:
        assert(grad_cliping_norm > 0)
        model.compile(Adam(adam_r, clipnorm=grad_cliping_norm), loss = 'binary_crossentropy', metrics = [])

    else:
        model.compile(Adam(adam_r), loss = 'binary_crossentropy', metrics = [])
    print('model total params:', model.count_params())
    
    return model


class CrossLayer(layers.Layer):
    def __init__(self, output_dim, num_layer, **kwargs):
        self.output_dim = output_dim
        self.num_layer = num_layer
        super(CrossLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.input_dim = input_shape[2]
        self.W = []
        self.bias = []
        for i in range(self.num_layer):
            self.W.append(self.add_weight(shape = [1, self.input_dim], initializer=keras.initializers.glorot_uniform(seed=fnl_seed),
                                          name = 'w_' + str(i), trainable = True))
            self.bias.append(self.add_weight(shape = [1, self.input_dim], initializer = 'zeros', name = 'b_' + str(i), trainable = True))
        self.built = True

    def call(self, inputs):
        for i in range(self.num_layer):
            if i == 0:
#                 cross = Lambda(lambda x: Add()([K.sum(self.W[i] * K.batch_dot(K.reshape(x, (-1, self.input_dim, 1)), x), 1, keepdims = True), self.bias[i], x]))(input)
                cross = Lambda(lambda x: Add()([tf.transpose(K.sum(self.W[i] * K.batch_dot(K.reshape(x, (-1, self.input_dim, 1)), x), 2, keepdims = True), perm=[0,2,1]), self.bias[i], x]))(inputs)  
            else:
                cross = Lambda(lambda x: Add()([tf.transpose(K.sum(self.W[i] * K.batch_dot(K.reshape(x, (-1, self.input_dim, 1)), inputs), 2, keepdims = True), perm=[0,2,1]), self.bias[i], inputs]))(cross)
        return Flatten()(cross)

    def compute_output_shape(self, input_shape):
        return (None, self.output_dim) 

## Data

### Split data

In [9]:
user = pd.read_csv('../userFeature.csv')
user.shape

(11420039, 24)

In [10]:
# 92%用户出现 1 次以上， 0.1%用户出现 1 次以上
train = pd.read_csv('train_kflod.csv')
train = train.reset_index(drop=False)
train.rename_axis({'index':'data_index'}, axis=1, inplace=True)

train.shape

(8798814, 157)

In [11]:
test = pd.read_csv('test_kflod_new_b.csv')

test.shape

(2265989, 156)

In [12]:
df = pd.concat([train, test])

df.shape

# del train
# del test
# gc.collect()

(11064803, 157)

In [11]:
# full_col = ['carrier_pos_max','carrier_min','gender_pos_min','interest1_feature_count_score','carrier','carrier_feature_count_score','kw1_feature_pos_count_score','interest2_feature_count_score','appIdAction_split_pos_count_score','education_max','ct_feature_count_score','interest2_feature_pos_count_score','topic3_feature_count_score','house_pos_mean','LBS_mean_sub','education','kw1_split_pos_count_score','os_split_pos_count_score','appIdInstall_split_pos_count_score','interest4_split_count_score','interest5_split_pos_count_score','LBS_pos_mean','topic2_feature_pos_count_score','education_std','carrier_std','consumptionAbility_pos_std','gender_std','interest2_split_pos_count_score','age_std','kw2_split_pos_count_score','gender_mean_sub','LBS_pos_std','topic3_split_pos_count_score','consumptionAbility_max','carrier_max','LBS_feature_count_score','consumptionAbility_mean','house_pos_max','appIdAction_feature_pos_count_score','LBS_min','consumptionAbility_mean_pos_sub','consumptionAbility_feature_count_score','house_mean','age_mean','house_pos_min','house_pos_std','LBS_mean_pos_sub','interest1_feature_pos_count_score','ct_split_count_score','topic2_split_pos_count_score','topic2_feature_count_score','topic1_split_count_score','age_feature_pos_count_score','aid','interest3_split_count_score','kw3_split_count_score','house_max','interest5_feature_count_score','kw2_feature_count_score','LBS_std','house_mean_sub','LBS_mean','kw3_split_pos_count_score','topic1_split_pos_count_score','interest1_split_pos_count_score','gender','gender_pos_mean','gender_feature_count_score','appIdAction_feature_count_score','carrier_mean_pos_sub','interest3_split_pos_count_score','age_pos_min','education_pos_mean','house_std','age_mean_sub','marriageStatus_feature_count_score','appIdInstall_feature_pos_count_score','house','education_pos_max','consumptionAbility_pos_min','age_feature_count_score','education_mean_pos_sub','interest3_feature_count_score','age_mean_pos_sub','gender_pos_max','uid','kw2_split_count_score','topic2_split_count_score','education_feature_count_score','interest4_feature_pos_count_score','age_max','carrier_pos_min','gender_feature_pos_count_score','consumptionAbility_std','education_pos_min','interest4_split_pos_count_score','consumptionAbility_pos_mean','interest5_split_count_score','os_feature_count_score','os_split_count_score','label','appIdInstall_split_count_score','LBS_pos_max','kw3_feature_count_score','topic3_split_count_score','topic1_feature_pos_count_score','education_pos_std','LBS_pos_min','carrier_pos_mean','topic1_feature_count_score','education_min','carrier_mean','education_mean','kw2_feature_pos_count_score','consumptionAbility_pos_max','consumptionAbility_mean_sub','gender_pos_std','age_min','interest4_feature_count_score','ct_split_pos_count_score','LBS','gender_min','marriageStatus_split_pos_count_score','gender_mean_pos_sub','carrier_feature_pos_count_score','topic3_feature_pos_count_score','interest5_feature_pos_count_score','age_pos_max','appIdAction_split_count_score','LBS_max','kw1_feature_count_score','marriageStatus_split_count_score','education_mean_sub','consumptionAbility_min','marriageStatus_feature_pos_count_score','consumptionAbility_feature_pos_count_score','appIdInstall_feature_count_score','LBS_feature_pos_count_score','interest3_feature_pos_count_score','age','os_feature_pos_count_score','interest1_split_count_score','kw1_split_count_score','kw3_feature_pos_count_score','ct_feature_pos_count_score','age_pos_std','carrier_pos_std','gender_mean','gender_max','age_pos_mean','house_mean_pos_sub','house_min','education_feature_pos_count_score','consumptionAbility','interest2_split_count_score','carrier_mean_sub']

# test1 = pd.read_csv('test_kflod0', usecols=full_col)
# test2 = pd.read_csv('test_kflod1', usecols=full_col)
# test3 = pd.read_csv('test_kflod2', usecols=full_col)
# test4 = pd.read_csv('test_kflod3', usecols=full_col)
# test5 = pd.read_csv('test_kflod4', usecols=full_col)

# test1.shape
# test2.shape
# test3.shape
# test4.shape
# test5.shape

# test = test1 + test2 + test3 + test4 + test5
# test = test / 5

# test.shape

# test.to_csv('test_kflod_new_b.csv', index=None)

(2265989, 156)

(2265989, 156)

(2265989, 156)

(2265989, 156)

(2265989, 156)

(2265989, 156)

In [11]:
# from sklearn.model_selection import StratifiedKFold
# skf = StratifiedKFold(n_splits=5, random_state=2018)

# ad_feature=pd.read_csv(RAW_FILE_PATH + 'adFeature.csv', usecols=['aid'])

# train_index, val_index = [], []
# for crt_aid in ad_feature['aid'].values:
#     crt_train = train[train['aid']==crt_aid].copy()
#     crt_train.reset_index(drop=True, inplace=True)
#     for crt_train_index, crt_val_index in skf.split(crt_train, crt_train['label']):
#         train_index += crt_train.iloc[crt_train_index]['data_index'].values.tolist()
#         val_index += crt_train.iloc[crt_val_index]['data_index'].values.tolist()
#         break
# #     break
# print(len(train_index), len(val_index), len(set(train_index) & set(val_index)), 
#       len(set(range(train.shape[0])) - set(train_index) - set(val_index)))

# del ad_feature
# del train

In [12]:
# pickle.dump(train_index, open('train_train_data_index.p', 'wb'))
# pickle.dump(val_index, open('train_val_data_index.p', 'wb'))

In [10]:
# train_index = pickle.load(open('train_train_data_index.p', 'rb'))
# val_index = pickle.load(open('train_val_data_index.p', 'rb'))

### Format Data

In [13]:
split_col = ['marriageStatus', 'interest1', 'interest2', 'interest3', 'interest4', 'interest5', 'kw1', 'kw2', 'kw3',
            'topic1', 'topic2', 'topic3', 'appIdInstall', 'appIdAction', 'ct', 'os']
feature_col = ['age', 'gender', 'marriageStatus', 'education', 'consumptionAbility', 'LBS', 'interest1', 'interest2',
               'interest3', 'interest4', 'interest5', 'kw1', 'kw2', 'kw3', 'topic1', 'topic2', 'topic3', 'appIdInstall',
               'appIdAction', 'ct', 'os', 'carrier']
single_col = ['age', 'gender', 'education', 'consumptionAbility', 'LBS', 'carrier','house']

In [11]:
def merge_df(data, file_path, on_fea):
    print('merge', file_path)
    new_userFeature = pd.read_csv(file_path, nrows=1)
    assert(on_fea in new_userFeature.columns)
    
    read_type_dict = {}
    for col in new_userFeature.columns[new_userFeature.dtypes == np.float64]:
        read_type_dict[col] = np.float32
    new_userFeature = pd.read_csv(file_path, dtype=read_type_dict)
    print('new feas shape:', new_userFeature.shape)
    print('before merge shape:', data.shape)
    data = data.merge(new_userFeature, on=on_fea, how='left')
    print('after merge shape:', data.shape)
    del new_userFeature
    return data

def format_data(data, is_fill_nan=False, read_file_oncols = [], is_load_basic_cols = True):
    print(data.shape)
    
    if is_load_basic_cols:
        if is_fill_nan:
            data = merge_df(data, './input/user_base_feas.csv', 'uid') #['uid', 'age', 'gender', 'education', 'consumptionAbility', 'LBS', 'carrier', 'house']
        else:
            data = merge_df(data, './input/user_base_feas_nofill.csv', 'uid') #['uid', 'age', 'gender', 'education', 'consumptionAbility', 'LBS', 'carrier', 'house']   

        data = merge_df(data, './input/user_multi_label.csv', 'uid') # 'marriageStatus', 'ct', 'os': label encoder
        data = merge_df(data, '../data/adFeature.csv', 'aid') 
  
    for file_path, on_col, _ in read_file_oncols:
        data = merge_df(data, file_path, on_col)    

    gc.collect()
    return data

def concat_data(data, file_path, exclude_cols, skip_nrows=0, nrows=None, verbose=False):
    new_feas = read_df_float32(file_path, exclude_cols=exclude_cols, nrows=nrows, skip_nrows=skip_nrows, 
                               verbose=verbose)
    assert(new_feas.shape[0] == data.shape[0])
    data = pd.concat([data, new_feas], axis=1)
    del new_feas
    return data

#### 分批处理

In [12]:
def build_input_data(data, concat_files, suffix, verbose=False, exclude_cols=[], nrows=None, skip_nrows=None):
    if verbose: print('data.shape:', data.shape)
    for file_path in concat_files:
        file_path = './nn_input'+file_path[file_path.rfind('/'):file_path.rfind('.')]+suffix
        data = concat_data(data, file_path, exclude_cols=exclude_cols, verbose=verbose, nrows=nrows,
                          skip_nrows=skip_nrows)
        if verbose: print('data.shape:', data.shape)
    return data

In [13]:
# 92%用户出现 1 次以上， 0.1%用户出现 1 次以上
train = pd.read_csv(RAW_FILE_PATH + 'train.csv')
train = train.reset_index(drop=False)
train.rename_axis({'index':'data_index'}, axis=1, inplace=True)
train.loc[train['label'] == -1, 'label'] = 0

exclude_cols = train.columns.tolist()

print(train.shape)

/home/user/cuser/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Using 'rename_axis' to alter labels is deprecated. Use '.rename' instead
  after removing the cwd from sys.path.


(8798814, 4)


In [14]:
train_val = train[train['data_index'].isin(val_index)]
train_val.reset_index(drop=True, inplace=True)

train_train = train[train['data_index'].isin(train_index)]
train_train.reset_index(drop=True, inplace=True)

print(train_val.shape, train_train.shape)
del train

(1759904, 4) (7038910, 4)


In [15]:
# 读取 离散值 的字典
cad_cate_feas = ['advertiserId', 'campaignId','creativeId', 'creativeSize', 'adCategoryId', 'productId',
               'productType', 'age', 'gender', 'marriageStatus', 'education', 'consumptionAbility', 'LBS', 
                'ct', 'os', 'carrier', 'house', 'aid']

cate_cnt_dict = {}
for fea in cad_cate_feas:
    tmp_dict = pickle.load(open('./transform_tool/'+fea+'.dict', 'rb'))
    cate_cnt_dict[fea] = len(tmp_dict)
    del tmp_dict

In [21]:
# print('train_val.shape:', train_val.shape)
# for file_path in concat_file:
#     file_path = './nn_input'+file_path[file_path.rfind('/'):file_path.rfind('.')]+'_train_val.csv'
#     train_val = concat_data(train_val, file_path, exclude_cols=exclude_cols, verbose=True)
#     print('train_val.shape:', train_val.shape)

In [22]:
# print(train_val.shape)
# print(train_val.columns)

In [23]:
# for fea in cate_feas:
#     uniq = full[fea].unique()
#     mapping = dict(zip(uniq,range(1,len(uniq) + 1)))
#     full[fea] = full[fea].map(mapping)

# print(full.isnull().sum().sum())
# train = full[:-test_len]
# print(train.shape)

In [24]:
# 对离散值重新编码，不然不能 embedding
def encode_feature(values):
    uniq = values.unique()
    mapping = dict(zip(uniq,range(1,len(uniq) + 1)))
    return values.map(mapping)

for fea in cate_feas:
    full[fea] = encode_feature(full[fea])

### DCN Train

In [25]:
# # tmp = pd.read_csv('./nn_input/user_base_feas_train_trian.csv')
# # tmp = pd.read_csv('./nn_input/user_multi_label_train_trian.csv')
# tmp = pd.read_csv('./nn_input/adFeature_train_trian.csv')
# tmp[tmp['data_index'].isin([2759904, 2759905, 2759906])].T

In [16]:
aid_dict = pickle.load(open('./transform_tool/aid.dict', 'rb'))
TRAIN_LEN, TEST_LEN = 8798814, 2265989

In [17]:
def train_model(model, train_val, is_adjust_params, adjust_params_val_len=500000, adjust_params_batch_cnt=2, 
                eopch_rows = 1000000, epoch_cnt=100, is_train_with_val=False, batch_size=1024, ):
    max_auc, crt_auc = 0.5, 0.5
    times = len(train_index) // eopch_rows + 1
    times = min(adjust_params_batch_cnt, times) if is_adjust_params else times
    train_val['aid'] = train_val['aid'].map(aid_dict)
    
#     is_adjust_params #验证集只取100000，只用一个batch训练，用 nomal_auc 决定是否继续下一轮训练
    if not is_train_with_val:
        if is_adjust_params:
            validation_data=([train_val[:adjust_params_val_len][c].values for c in use_feas], 
                             train_val[:adjust_params_val_len]['label'].astype(int).values.reshape(-1, 1))
        else:
            validation_data=([train_val[c].values for c in use_feas], 
                             train_val['label'].astype(int).values.reshape(-1, 1))


    for crt_epoch in range(1, epoch_cnt+1): # epoch
        if is_train_with_val:
            print('training with val')
            model.fit([train_val[c].values for c in use_feas], train_val['label'].astype(int).values.reshape(-1, 1),
                  batch_size=512, epochs=1, shuffle=True)
        for crt_time in range(times): #batch
            print('epoch:', crt_epoch, ', batch:', crt_time+1)
            batch_start, batch_end = crt_time*eopch_rows, (crt_time+1)*eopch_rows
            df_batch = train_train[batch_start:batch_end].copy()
            df_batch.reset_index(drop=True, inplace=True)
            print('before concat shape:', df_batch.shape)
            df_batch = build_input_data(df_batch, concat_file, suffix='_train_train.csv', exclude_cols=exclude_cols, 
                                        nrows=eopch_rows, verbose=False, 
                                        skip_nrows=list(range(1, batch_start+1)) + list(range(batch_end+1,TRAIN_LEN+TEST_LEN+1)))
            df_batch['aid'] = df_batch['aid'].map(aid_dict)
            print('after concat shape:', df_batch.shape)

            model.fit([df_batch[c].values for c in use_feas], df_batch['label'].astype(int).values.reshape(-1, 1),
                  batch_size=batch_size, epochs=1, shuffle=True)
        
                
        if not is_train_with_val:
            pred = model.predict(validation_data[0])
            normal_crt_auc = roc_auc_score(validation_data[1], pred)

            if is_adjust_params: 
                crt_auc = cal_tecent_auc(train_val[:adjust_params_val_len][['aid', 'uid', 'label']], pred)
            else:
                crt_auc = cal_tecent_auc(train_val[['aid', 'uid', 'label']], pred)

            crt_log_loss = log_loss(validation_data[1], pred)
            print('x'*10, 'epoch', crt_epoch,',t_auc =', crt_auc, ',normal_auc =', normal_crt_auc, ', log_loss =', crt_log_loss, 'x'*10)
            if crt_auc<=max_auc:
                print('early stopping,best epoch =', crt_epoch-1)
                break
            else:
                max_auc = crt_auc
                
    if not is_train_with_val:
        return model, crt_epoch-1, max_auc 
    else:
        pred = model.predict([train_val[c].values for c in use_feas])
        normal_crt_auc = roc_auc_score(train_val['label'].astype(int).values.reshape(-1, 1), pred)
        crt_auc = cal_tecent_auc(train_val[['aid', 'uid', 'label']], pred)
        print('refit, t_auc={}, normal_auc'.format(crt_auc, normal_crt_auc))
        return model, crt_epoch, crt_auc

In [28]:
# train_val = train_val[exclude_cols]
# train_val.shape

In [29]:
# {'interest1': [2, 0.6741602659004512], 'interest2': [4, 0.6881776929214258], 'interest3': [1, 0.668021058741138], 'interest4': [3, 0.678914592403243], 'interest5': [2, 0.6749052878005422], 'kw1': [1, 0.6579797710596459], 'kw2': [5, 0.7308447387147256], 'kw3': [2, 0.666857688761546], 'topic1': [1, 0.6621039093655526], 'topic2': [1, 0.6882925237880755], 'topic3': [4, 0.6790555436246852], 'appIdAction': [1, 0.6619925236093374], 'appIdInstall': [4, 0.6767158121418987]}

In [30]:
# # 特征筛选

# watch_cols = [
# #     'interest1', 'interest2', 'interest3', 'interest4', 'interest5',
# # 'kw1', 'kw2', 'kw3', 'topic1', 'topic2', 'topic3', 'appIdAction', 'appIdInstall',
#  'interest1_user', 'interest2_user', 'interest3_user', 'interest4_user', 'interest5_user',
# 'kw1_user', 'kw2_user', 'kw3_user', 'topic1_user', 'topic2_user', 'topic3_user', 
#  'appIdAction_user', 'appIdInstall_user'
# ]
# # TODO cos_similarity
# watch_rst_dict = {}

# for watch_col in watch_cols:
#     print('x'*10, 'watching', watch_col, 'x'*10)
#     use_embedding_name = [watch_col, watch_col+'_neg', watch_col+'_pos']
#     use_file_name = use_embedding_name.copy()
#     concat_file = [embedding_file_dict[x] for x in use_file_name]
#     concat_file = ['./nn_input'+file_path[0][file_path[0].rfind('/'):] for file_path in concat_file]
#     concat_file = ['./nn_input/user_base_feas.csv', './nn_input/user_multi_label.csv', './nn_input/adFeature.csv'] + concat_file
#     # concat_file = ['./nn_input/user_base_feas.csv', './nn_input/user_multi_label.csv', './nn_input/adFeature.csv']


#     conti_feas = []
#     for name in use_embedding_name:
#         conti_feas += [name+'_'+str(i) for i in range(embedding_file_dict[name][2])]
            
#     cate_feas = cad_cate_feas
#     use_feas = cate_feas + conti_feas
#     print('(use_feas, cate_feas, conti_feas) len :', len(use_feas), len(cate_feas), len(conti_feas))
    
    
#     print('train_val before concat shape:', train_val.shape)
#     train_val = build_input_data(train_val, concat_file, suffix='_train_val.csv', 
#                                  exclude_cols=exclude_cols, verbose=False)
#     print('train_val after concat shape:', train_val.shape)

    

#     K.clear_session()
#     gc.collect()

#     inp_layer, inp_embed = feature_generate(cate_cnt_dict, cate_feas, conti_feas)
#     model = fit(inp_layer, inp_embed, wide_feas_cnt=-1,# len(wide_feas)
#                 deep_layer_sizes=[2**8, 2**8], cross_layer_cnt=1, adam_r=0.005,
#                 drop_out_r = 0.5, use_bn=True , is_print_sum=False, grad_cliping_norm=None)
#     model, best_epoch, best_score = train_model(model, is_adjust_params=True, train_val=train_val.copy())
#     watch_rst_dict[watch_col] = [best_epoch, best_score]
    
#     train_val = train_val[exclude_cols]

In [18]:
watch_rst_dict = {'interest1': [2, 0.6741602659004512], 'interest2': [4, 0.6881776929214258], 'interest3': [1, 0.668021058741138], 'interest4': [3, 0.678914592403243], 'interest5': [2, 0.6749052878005422], 'kw1': [1, 0.6579797710596459], 'kw2': [5, 0.7308447387147256], 'kw3': [2, 0.666857688761546], 'topic1': [1, 0.6621039093655526], 'topic2': [1, 0.6882925237880755], 'topic3': [4, 0.6790555436246852], 'appIdAction': [1, 0.6619925236093374], 'appIdInstall': [4, 0.6767158121418987],
                 'interest1_user': [1, 0.6627513010954947], 'interest2_user': [1, 0.7066595334293738], 'interest3_user': [1, 0.6659990220191173], 'interest4_user': [2, 0.6682035487581718], 'interest5_user': [5, 0.6980055922867054], 'kw1_user': [1, 0.6567042806276884], 'kw2_user': [3, 0.7100131247991951], 'kw3_user': [3, 0.6704050863946275], 'topic1_user': [1, 0.6642590145792624], 'topic2_user': [1, 0.6799091341666444], 'topic3_user': [2, 0.6698498902665675], 'appIdAction_user': [3, 0.6642784154265863], 'appIdInstall_user': [2, 0.6672428986143043]}
watch_rst_item = sorted(watch_rst_dict.items(), key=lambda x:x[1][1], reverse=True)

In [19]:
[item[0] for item in watch_rst_item[:20]]

['kw2',
 'kw2_user',
 'interest2_user',
 'interest5_user',
 'topic2',
 'interest2',
 'topic2_user',
 'topic3',
 'interest4',
 'appIdInstall',
 'interest5',
 'interest1',
 'kw3_user',
 'topic3_user',
 'interest4_user',
 'interest3',
 'appIdInstall_user',
 'kw3',
 'interest3_user',
 'appIdAction_user']

In [ ]:
# use_cols = [
# #     'interest1', 'interest2', 'interest3', 'interest4', 'interest5',
# # 'kw1', 'kw2', 'kw3', 'topic1', 'topic2', 'topic3', 'appIdAction', 'appIdInstall',
#  'interest1_user', 'interest2_user', 'interest3_user', 'interest4_user', 'interest5_user',
# 'kw1_user', 'kw2_user', 'kw3_user', 'topic1_user', 'topic2_user', 'topic3_user', 
#  'appIdAction_user', 'appIdInstall_user'
# ]


# use_cols = [item[0] for item in watch_rst_item[:3]]
use_cols = ['kw2', 'interest2_user', 'interest5_user', 'topic2', 'topic3', 'interest4', 'appIdInstall',
           'interest5', 'interest1']
use_embedding_name = []
print('x'*10, 'add', use_cols, 'x'*10)
for use_col in use_cols:
#     use_embedding_name += [use_col, use_col+'_neg', use_col+'_pos']
    use_embedding_name += [use_col, use_col+'_pos']
    
    

use_file_name = use_embedding_name.copy()
concat_file = [embedding_file_dict[x] for x in use_file_name]
concat_file = ['./nn_input'+file_path[0][file_path[0].rfind('/'):] for file_path in concat_file]
concat_file = ['./nn_input/user_base_feas.csv', './nn_input/user_multi_label.csv', './nn_input/adFeature.csv'] + concat_file
concat_file += ['./nn_input/full_cos_similarity_part1.csv', './nn_input/full_cos_similarity_part2.csv']


conti_feas = []
for name in use_embedding_name:
    conti_feas += [name+'_'+str(i) for i in range(embedding_file_dict[name][2])]
wide_feas = ['interest1_pos_cos', 'interest1_neg_cos',
       'interest2_pos_cos', 'interest2_neg_cos', 'interest3_pos_cos',
       'interest3_neg_cos', 'interest4_pos_cos', 'interest4_neg_cos',
       'interest5_pos_cos', 'interest5_neg_cos', 'kw1_pos_cos', 'kw1_neg_cos',
       'kw2_pos_cos', 'kw2_neg_cos', 'kw3_pos_cos', 'kw3_neg_cos',
       'topic1_pos_cos', 'topic1_neg_cos', 'topic2_pos_cos', 'topic2_neg_cos',
       'topic3_pos_cos', 'topic3_neg_cos', 'appIdAction_pos_cos',
       'appIdAction_neg_cos', 'appIdInstall_pos_cos', 'appIdInstall_neg_cos',
       'interest1_user_pos_cos', 'interest1_user_neg_cos',
       'interest2_user_pos_cos', 'interest2_user_neg_cos',
       'interest3_user_pos_cos', 'interest3_user_neg_cos',
       'interest4_user_pos_cos', 'interest4_user_neg_cos',
       'interest5_user_pos_cos', 'interest5_user_neg_cos', 'kw1_user_pos_cos',
       'kw1_user_neg_cos', 'kw2_user_pos_cos', 'kw2_user_neg_cos',
       'kw3_user_pos_cos', 'kw3_user_neg_cos', 'topic1_user_pos_cos',
       'topic1_user_neg_cos', 'topic2_user_pos_cos', 'topic2_user_neg_cos',
       'topic3_user_pos_cos', 'topic3_user_neg_cos',
       'appIdAction_user_pos_cos', 'appIdAction_user_neg_cos',
       'appIdInstall_user_pos_cos', 'appIdInstall_user_neg_cos']
conti_feas += wide_feas
    
cate_feas = cad_cate_feas
use_feas = cate_feas + conti_feas
use_feas += wide_feas

print('(use_feas, cate_feas, conti_feas) len :', len(use_feas), len(cate_feas), len(conti_feas), len(wide_feas))


train_val = train_val[exclude_cols]
print('train_val before concat shape:', train_val.shape)
train_val = build_input_data(train_val, concat_file, suffix='_train_val.csv', 
                             exclude_cols=exclude_cols, verbose=False)
print('train_val after concat shape:', train_val.shape)


# #调参
# print('adjusting params')
# K.clear_session()
# gc.collect()
# inp_layer, inp_embed = feature_generate(cate_cnt_dict, cate_feas, conti_feas, wide_feas)
# model = fit(inp_layer, inp_embed, wide_feas_cnt=len(wide_feas),# len(wide_feas) or -1
#             deep_layer_sizes=[2**10, 2**10], cross_layer_cnt=0, adam_r=0.01,
#             drop_out_r = 0.5, use_bn=True , is_print_sum=False, grad_cliping_norm=None)



# model, best_epoch, best_score = train_model(model, is_adjust_params=True, train_val=train_val.copy(), epoch_cnt=2, 
#                                             adjust_params_val_len=1000000, adjust_params_batch_cnt=1, 
#                 eopch_rows = 1000000, is_train_with_val=False, batch_size=1024, )
# print('best_epoch={}, best_score={}'.format(best_epoch, best_score))
    
    
# 为了确定最佳迭代次数
# K.clear_session()
# gc.collect()
# inp_layer, inp_embed = feature_generate(cate_cnt_dict, cate_feas, conti_feas, wide_feas)
# model = fit(inp_layer, inp_embed, wide_feas_cnt=len(wide_feas),# len(wide_feas) or -1
#             deep_layer_sizes=[2**10, 2**10], cross_layer_cnt=0, adam_r=0.01,
#             drop_out_r = 0.5, use_bn=True , is_print_sum=False, grad_cliping_norm=None)

# model, best_epoch, best_score = train_model(model, is_adjust_params=True, train_val=train_val.copy(), epoch_cnt=100, 
#                                             adjust_params_val_len=1000000, adjust_params_batch_cnt=1, 
#                 eopch_rows = 1000000, is_train_with_val=False, batch_size=1024, )
# print('best_epoch={}, best_score={}'.format(best_epoch, best_score))
# model.save('./model/mid_dcn_3best.model')



# # 为了生成提交模型
print('training submition model')
K.clear_session()
gc.collect()
inp_layer, inp_embed = feature_generate(cate_cnt_dict, cate_feas, conti_feas, wide_feas)
model = fit(inp_layer, inp_embed, wide_feas_cnt=len(wide_feas),# len(wide_feas) or -1
            deep_layer_sizes=[2**10, 2**10], cross_layer_cnt=0, adam_r=0.01,
            drop_out_r = 0.5, use_bn=True , is_print_sum=False, grad_cliping_norm=None)
model, best_epoch, best_score = train_model(model, is_adjust_params=False, train_val=train_val.copy(), epoch_cnt=4, 
                                            is_train_with_val=True, batch_size=1024)
print('best_epoch={}, best_score={}'.format(best_epoch, best_score))
model.save('./model/dcn_4best.model')

xxxxxxxxxx add ['kw2', 'interest2_user', 'interest5_user', 'topic2', 'topic3', 'interest4', 'appIdInstall', 'interest5', 'interest1'] xxxxxxxxxx
(use_feas, cate_feas, conti_feas) len : 546 18 476 52
training submition model
deep_layer_sizes: [1024, 1024] , cross_layer_cnt: 0 , adam_r: 0.01 , drop_out_r: 0.5 , use_bn: True , grad_cliping_norm: None
int_shape(input) : (None, 1, 686)


/home/user/cuser/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  del sys.path[0]
/home/user/cuser/anaconda3/lib/python3.6/site-packages/keras/legacy/layers.py:465: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


model total params: 1805705
training with val
Epoch 1/1
1759904/1759904 [==============================] - 546s 310us/step - loss: 0.1915
epoch: 1 , batch: 1
before concat shape: (1000000, 4)


In [ ]:
[2**10, 2**10], cl=1
epoch 2 ,t_auc = 0.7289001972743767 ,normal_auc = 0.7118184878598045 , log_loss = 0.17652163961791237

将cos作为wide feas
epoch 2 ,t_auc = 0.7325104221177201 ,normal_auc = 0.7145309378586567 , log_loss = 0.17599003961073162

cl=0
epoch 2 ,t_auc = 0.7346608708161895 ,normal_auc = 0.7192211249427258 , log_loss = 0.17414765849121056 best

cl=2
epoch 2 ,t_auc = 0.7289632454744969 ,normal_auc = 0.7074278745342352 , log_loss = 0.17691881454519695


调DNN参数
[2**9]*2
epoch 2 ,t_auc = 0.7344369363928379 ,normal_auc = 0.719321657492843 , log_loss = 0.17471447077338997

[2**11]*2
epoch 2 ,t_auc = 0.7307082278364945 ,normal_auc = 0.7076002244158689

just DNN, no wide
epoch 2 ,t_auc = 0.7081105214024801 ,normal_auc = 0.6794491426432617 , log_loss = nan

add neg (bad)
epoch 2 ,t_auc = 0.7325739793664645 ,normal_auc = 0.7188239715791924


[2**10, 2**10, 2**8], cl=0
epoch 2 ,t_auc = 0.7320133672675392 ,normal_auc = 0.7180290353534401 , log_loss = 0.1745249943555784

# 加其他特征

In [108]:
# full_cos_similarity_train_train 有问题
read_df_float32('./nn_input/full_cos_similarity_train_train.csv', nrows=5)

,interest1_pos_cos,interest1_neg_cos,interest2_pos_cos,interest2_neg_cos,interest3_pos_cos,interest3_neg_cos,interest4_pos_cos,interest4_neg_cos,interest5_pos_cos,interest5_neg_cos,...,topic3_pos_cos,topic3_neg_cos,appIdAction_pos_cos,appIdAction_neg_cos,appIdInstall_pos_cos,appIdInstall_neg_cos,uid,aid,data_index,label
0,0.170512,0.159191,-0.237623,0.333116,0.038639,0.038099,0.072134,0.071311,0.168434,0.255436,...,0.211015,0.209807,0.120582,0.119992,0.141238,0.14088,37197874,977,1479136,1.0
1,0.363074,0.362413,0.992726,-0.233101,0.038639,0.038099,0.072134,0.071311,0.299767,-0.190183,...,0.211015,0.209807,0.120582,0.119992,0.141238,0.14088,15885390,1507,1489084,1.0
2,0.363074,0.362413,0.094020,-0.650120,0.038639,0.038099,0.072134,0.071311,1.282168,1.277390,...,0.211015,0.209807,0.120582,0.119992,0.141238,0.14088,28840700,1507,1490536,1.0
3,0.363074,0.362413,0.255975,-0.615731,0.038639,0.038099,0.072134,0.071311,1.282168,1.277390,...,0.211015,0.209807,0.120582,0.119992,0.141238,0.14088,24394229,1507,1503375,1.0
4,0.316870,0.319905,0.150336,-0.262766,0.038639,0.038099,0.072134,0.071311,1.282168,1.277390,...,0.211015,0.209807,0.120582,0.119992,0.141238,0.14088,81456510,1449,1504692,1.0


In [111]:
read_df_float32('./nn_input/full_cos_similarity_train_val.csv', nrows=5)

,interest1_pos_cos,interest1_neg_cos,interest2_pos_cos,interest2_neg_cos,interest3_pos_cos,interest3_neg_cos,interest4_pos_cos,interest4_neg_cos,interest5_pos_cos,interest5_neg_cos,...,topic3_pos_cos,topic3_neg_cos,appIdAction_pos_cos,appIdAction_neg_cos,appIdInstall_pos_cos,appIdInstall_neg_cos,uid,aid,data_index,label
0,0.275686,0.278757,1.221731,1.234648,0.038639,0.038099,0.072134,0.071311,1.282168,1.277390,...,0.211015,0.209807,0.120582,0.119992,0.141238,0.14088,78508957,699,0,0.0
1,0.299687,0.297473,1.221731,1.234648,0.038639,0.038099,0.072134,0.071311,1.030373,0.830332,...,0.211015,0.209807,0.120582,0.119992,0.141238,0.14088,3637295,1991,1,0.0
2,0.329802,0.329046,1.221731,1.234648,0.038639,0.038099,0.072134,0.071311,1.282168,1.277390,...,0.211015,0.209807,0.120582,0.119992,0.141238,0.14088,19229018,1119,2,0.0
3,-0.254887,-0.263144,0.098758,0.004091,0.038639,0.038099,0.072134,0.071311,-1.006416,-1.044801,...,0.211015,0.209807,0.120582,0.119992,0.141238,0.14088,79277120,2013,3,0.0
4,0.276430,0.272662,0.340303,-0.072609,0.038639,0.038099,0.072134,0.071311,-0.587941,-0.511628,...,0.211015,0.209807,0.120582,0.119992,0.141238,0.14088,41528441,692,4,0.0


In [113]:
read_df_float32('./nn_input/full_cos_similarity_test.csv', nrows=50)

,interest1_pos_cos,interest1_neg_cos,interest2_pos_cos,interest2_neg_cos,interest3_pos_cos,interest3_neg_cos,interest4_pos_cos,interest4_neg_cos,interest5_pos_cos,interest5_neg_cos,...,topic3_pos_cos,topic3_neg_cos,appIdAction_pos_cos,appIdAction_neg_cos,appIdInstall_pos_cos,appIdInstall_neg_cos,uid,aid,data_index,label
0,2.068078e-07,3.099270e-07,-1.731385e-08,-2.016560e-09,-0.000001,6.203366e-07,4.260101e-07,-2.211204e-07,-9.354349e-09,4.000788e-08,...,7.512642e-08,6.087221e-08,1.901087e-07,-2.693938e-07,-1.288679e-07,1.651050e-07,64355836,2118,0,NaN
1,2.068078e-07,3.099270e-07,-1.731385e-08,-2.016560e-09,-0.000001,6.203366e-07,4.260101e-07,-2.211204e-07,-9.354349e-09,4.000788e-08,...,7.512642e-08,6.087221e-08,1.901087e-07,-2.693938e-07,-1.288679e-07,1.651050e-07,45051997,692,1,NaN
2,2.068078e-07,3.099270e-07,-1.731385e-08,-2.016560e-09,-0.000001,6.203366e-07,4.260101e-07,-2.211204e-07,-9.354349e-09,4.000788e-08,...,7.512642e-08,6.087221e-08,1.901087e-07,-2.693938e-07,-1.288679e-07,1.651050e-07,10869198,692,2,NaN
3,2.068078e-07,3.099270e-07,-1.731385e-08,-2.016560e-09,-0.000001,6.203366e-07,4.260101e-07,-2.211204e-07,-9.354349e-09,4.000788e-08,...,7.512642e-08,6.087221e-08,1.901087e-07,-2.693938e-07,-1.288679e-07,1.651050e-07,75929554,1918,3,NaN
4,2.068078e-07,3.099270e-07,-1.731385e-08,-2.016560e-09,-0.000001,6.203366e-07,4.260101e-07,-2.211204e-07,-9.354349e-09,4.000788e-08,...,7.512642e-08,6.087221e-08,1.901087e-07,-2.693938e-07,-1.288679e-07,1.651050e-07,5790162,1596,4,NaN
5,2.068078e-07,3.099270e-07,-1.731385e-08,-2.016560e-09,-0.000001,6.203366e-07,4.260101e-07,-2.211204e-07,-9.354349e-09,4.000788e-08,...,7.512642e-08,6.087221e-08,1.901087e-07,-2.693938e-07,-1.288679e-07,1.651050e-07,14187856,1044,5,NaN
6,2.068078e-07,3.099270e-07,-1.731385e-08,-2.016560e-09,-0.000001,6.203366e-07,4.260101e-07,-2.211204e-07,-9.354349e-09,4.000788e-08,...,7.512642e-08,6.087221e-08,1.901087e-07,-2.693938e-07,-1.288679e-07,1.651050e-07,6387181,12,6,NaN
7,2.068078e-07,3.099270e-07,-1.731385e-08,-2.016560e-09,-0.000001,6.203366e-07,4.260101e-07,-2.211204e-07,-9.354349e-09,4.000788e-08,...,7.512642e-08,6.087221e-08,1.901087e-07,-2.693938e-07,-1.288679e-07,1.651050e-07,27283647,1044,7,NaN
8,2.068078e-07,3.099270e-07,-1.731385e-08,-2.016560e-09,-0.000001,6.203366e-07,4.260101e-07,-2.211204e-07,-9.354349e-09,4.000788e-08,...,7.512642e-08,6.087221e-08,1.901087e-07,-2.693938e-07,-1.288679e-07,1.651050e-07,30281845,916,8,NaN
9,2.068078e-07,3.099270e-07,-1.731385e-08,-2.016560e-09,-0.000001,6.203366e-07,4.260101e-07,-2.211204e-07,-9.354349e-09,4.000788e-08,...,7.512642e-08,6.087221e-08,1.901087e-07,-2.693938e-07,-1.288679e-07,1.651050e-07,4660794,692,9,NaN


In [114]:
read_df_float32('./input/full_cos_similarity.csv', nrows=5)

,aid,uid,interest1_pos_cos,interest1_neg_cos,interest2_pos_cos,interest2_neg_cos,interest3_pos_cos,interest3_neg_cos,interest4_pos_cos,interest4_neg_cos,...,topic1_pos_cos,topic1_neg_cos,topic2_pos_cos,topic2_neg_cos,topic3_pos_cos,topic3_neg_cos,appIdAction_pos_cos,appIdAction_neg_cos,appIdInstall_pos_cos,appIdInstall_neg_cos
0,699,78508957,0.992389,0.992715,1.000000,1.000000,1.0,1.0,1.0,1.0,...,-0.076818,-0.056763,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.0
1,1991,3637295,0.994479,0.994345,1.000000,1.000000,1.0,1.0,1.0,1.0,...,0.364368,0.332267,0.078147,0.070990,1.0,1.0,1.0,1.0,1.0,1.0
2,1119,19229018,0.997102,0.997094,1.000000,1.000000,1.0,1.0,1.0,1.0,...,-0.111505,-0.153256,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.0
3,2013,79277120,0.946178,0.945528,0.489374,0.453966,1.0,1.0,1.0,1.0,...,-0.013185,-0.084461,0.122349,0.031413,1.0,1.0,1.0,1.0,1.0,1.0
4,692,41528441,0.992454,0.992185,0.599207,0.419932,1.0,1.0,1.0,1.0,...,0.376273,0.323023,0.066619,0.100156,1.0,1.0,1.0,1.0,1.0,1.0


In [42]:
watch_layer_name = ['age_inp', 'embedding_1', 'merge_1','batch_normalization_1', 'flatten_1', 
                    'dense_1', 'batch_normalization_2', 'dense_2', 'dense_3']

### Predict

In [ ]:
print(train_val.columns.tolist())
train_val = train_val[exclude_cols]
train_val.shape

In [ ]:
# del train_val
gc.collect()

In [ ]:
%%time
test = read_df_float32(RAW_FILE_PATH+'test1.csv')
test = build_input_data(test, concat_file, suffix='_test.csv', 
                        exclude_cols=exclude_cols, verbose=False)
print(test.shape)
test.head()

In [47]:
# from multiprocessing import Pool

# def build_input_data_multipro(data, concat_files, suffix, verbose=False, exclude_cols=[], nrows=None, skip_nrows=None):
#     if verbose: print('data.shape:', data.shape)
#     pool = Pool(processes=8)
# #     file_path, read_cols=[], sep=',', nrows=None, exclude_cols=[], skip_nrows=None, verbose=False
#     read_files = ['./nn_input'+file_path[file_path.rfind('/'):file_path.rfind('.')]+suffix for file_path in concat_files]
    
#     df_list = pool.starmap(read_df_float32, zip(read_files, [[]]*len(read_files), [',']*len(read_files), [nrows]*len(read_files),
#                                             [exclude_cols]*len(read_files), [skip_nrows]*len(read_files), 
#                                             [verbose]*len(read_files)))
#     pool.close() 
#     pool.join()

    
# #     read_df_float32(file_path, exclude_cols=exclude_cols, nrows=nrows, skip_nrows=skip_nrows, 
# #                                verbose=verbose)
    
    
#     data = pd.concat(df_list, ignore_index=True)
#     return data

# # 5 2min 7s
# # 3 
# # 8

# %%time
# test = read_df_float32(RAW_FILE_PATH+'test1.csv')
# test = build_input_data_multipro(test, concat_file, suffix='_test.csv', 
#                         exclude_cols=exclude_cols, verbose=False)
# print(test.shape)
# test.head()

In [ ]:
pred = model.predict([test[c].values for c in use_feas])
print(len(pred))

In [ ]:
print(len(use_feas))
submit = test[['aid','uid']].copy()
submit['score'] = pred
submit['score'] = submit['score'].apply(lambda x: float('%.8f' % x))
print(submit.shape)
submit.head()

In [ ]:
sub_zip_name = 'dcn_best9_widecos'
submit.to_csv('submission.csv', index=False)
os.system('zip -j '+sub_zip_name+'.zip submission.csv')
os.system('mv submission.csv ./submit/a_a/'+sub_zip_name+'.csv')
os.system('mv '+sub_zip_name+'.zip ./submit/a_a/')
del sub_zip_name

### 中间层输出分析

In [32]:
def get_activations(model, model_inputs, print_shape_only=False, layer_name=None):
    print('----- activations -----')
    activations = []
    inp = model.input

    model_multi_inputs_cond = True
    if not isinstance(inp, list):
        # only one input! let's wrap it in a list.
        inp = [inp]
        model_multi_inputs_cond = False

    outputs = [layer.output for layer in model.layers if
               layer.name in layer_name or layer_name is None]  # all layer outputs
    output_names = [layer.name for layer in outputs]

    funcs = [K.function(inp + [K.learning_phase()], [out]) for out in outputs]  # evaluation functions

    if model_multi_inputs_cond:
        list_inputs = []
        list_inputs.extend(model_inputs)
        list_inputs.append(0.)
    else:
        list_inputs = [model_inputs, 0.]

    # Learning phase. 0 = Test mode (no dropout or batch normalization)
    # layer_outputs = [func([model_inputs, 0.])[0] for func in funcs]
    layer_outputs = [func(list_inputs)[0] for func in funcs]
    for name, layer_activations in zip(output_names, layer_outputs):
        activations.append(layer_activations)
        if print_shape_only:
            print('['+ name +'] \n', layer_activations.shape)
        else:
            print('['+ name +'] \n', layer_activations)
    return activations

In [61]:
# watch_layer_name = ['advertiserId_inp', 'embedding_1','flatten_1', 'dense_1', 'batch_normalization_1', 'dense_2']
test_X = train_val[:3][use_feas]

layer_output = get_activations(model, [np.reshape(test_X[c].values, (-1, 1)) for c in test_X.columns],
                               layer_name=watch_layer_name)

----- activations -----
[age_inp:0] 
 [[1]
 [2]
 [3]]
[embedding_1/Gather:0] 
 [[[-0.9250501   1.2410071   0.9853753   1.6281238   2.358317
    0.7735624   1.2173765  -2.1765685   1.9326898  -0.723503
   -0.8557621  -0.08398576 -0.80780953  1.0637301   1.1095247
    1.431616   -1.3574914 ]]

 [[-0.22646073  0.04168807  1.7829571   0.51483923  0.48427078
   -0.14020818  0.94762886 -0.5849288   0.82886815 -0.33185005
    0.09381137 -0.26539898  0.00909028  1.4508789  -0.07374161
   -0.09468328 -1.3837662 ]]

 [[ 1.1797806  -0.4187837  -2.0375638  -1.9235202  -1.7184542
   -0.25618544 -2.9839385   1.9926614  -2.3583672  -0.82218385
   -1.7309016   0.8810462   1.138139   -1.65847    -0.4909727
   -2.3574266   1.4330499 ]]]
[merge_1/concat:0] 
 [[[-9.25050080e-01  1.24100709e+00  9.85375285e-01  1.62812376e+00
    2.35831690e+00  7.73562372e-01  1.21737647e+00 -2.17656851e+00
    1.93268979e+00 -7.23502994e-01 -8.55762124e-01 -8.39857608e-02
   -8.07809532e-01  1.06373012e+00  1.10952473e+0

In [133]:
print(len(model.layers))
for layer in model.layers:
    if layer.name in watch_layer_name:
        print(layer.name, '\n', [list(x) for x in layer.get_weights()])
#         if layer.name == 'dense_1': break

42
age_inp 
 []
embedding_8 
 [[array([-0.03950969, -0.01801052, -0.01387911,  0.00423812, -0.00268453,
        0.04172214], dtype=float32), array([-0.02927061,  0.0395574 , -0.03495294,  0.00336866, -0.03537874,
       -0.00117604], dtype=float32), array([ 0.01197215, -0.0311771 ,  0.02833773,  0.04230175,  0.04902912,
       -0.00827999], dtype=float32), array([-0.03255901, -0.04889313,  0.02244487, -0.02943592,  0.04755808,
       -0.0169274 ], dtype=float32), array([-0.01195424,  0.02227953, -0.03338635,  0.04430771, -0.036684  ,
       -0.02294836], dtype=float32), array([-0.02750608, -0.01165203,  0.01704938, -0.04838009,  0.00898879,
       -0.01997925], dtype=float32)]]
batch_normalization_1 
 [[0.791457, 1.1080651, 0.95610124, 1.0412216, 0.6335143, 0.98002887, 0.7339721, 1.1092327, 1.1048548, 0.9857835, 1.2658647, 0.7251241, 0.814394, 1.2743741, 0.867069, 0.78844094, 1.1101295, 0.9999712, 0.82189196, 0.91067904, 1.0165347, 0.81271577, 0.9745142, 0.79428005, 1.0448805, 0.738302

In [28]:
# CV，未完成，可以通过运行完后 history.history 得到分数
# from sklearn.model_selection import StratifiedKFold
# skf = StratifiedKFold(n_splits=6, shuffle=True, random_state=2018)

# crt_fold = 1
# for train_ind, val_ind in skf.split(full[:-test_len], full[:-test_len]['is_trade']):
#     train, val = full.iloc[train_ind][use_feas], full.iloc[val_ind][use_feas]
#     train_y, val_y = full.iloc[train_ind]['is_trade'], full.iloc[val_ind]['is_trade']
    
#     print('crt_fold',crt_fold,'#'*30)
    
# #     train[conti_feas] = 
#     assert not np.any(np.isnan(train))
#     assert not np.any(np.isnan(val))
#     assert not np.any(np.isnan(train_y))
#     assert not np.any(np.isnan(val_y))
#     print(sum(train_y), sum(val_y)) #(3934.0, 787.0)
    
#     print(train.shape, val.shape)
#     crt_fold += 1
#     history = History()
#     model = fit(inp_layer, inp_embed, train, train_y.values.reshape(-1, 1),
#                 deep_layer_sizes=[2**8, 2**6], cross_layer_cnt=3, adam_r=0.001, batch_size=1024, epochs=50,
#                 drop_out_r = 1, use_cross_layer=True, is_print_sum=False, early_stopping=3, history=history,
#                 validation_data=([val[c] for c in val.columns], val_y.values.reshape(-1, 1)))
           
#     inp = model.input                                           # input placeholder
#     outputs = [layer.output for layer in model.layers]          # all layer outputs
#     functors = [K.function([inp]+ [K.learning_phase()], [out]) for out in outputs]  # evaluation functions

#     break

('crt_fold', 1, '##############################')
(3934.0, 787.0)
((217102, 45), (43421, 45))


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:16: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  app.launch_new_instance()
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:36: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


Train on 217102 samples, validate on 43421 samples
Epoch 1/50
217102/217102 [==============================] - 8s 38us/step - loss: 0.1613 - val_loss: 0.0918
Epoch 2/50
217102/217102 [==============================] - 7s 31us/step - loss: 0.0879 - val_loss: 0.0867
Epoch 3/50
217102/217102 [==============================] - 7s 31us/step - loss: 0.0820 - val_loss: 0.0884
Epoch 4/50
217102/217102 [==============================] - 7s 31us/step - loss: 0.0795 - val_loss: 0.0916
Epoch 5/50
217102/217102 [==============================] - 7s 30us/step - loss: 0.0787 - val_loss: 0.0906
